📦 **Imports and Data Load**


In [ ]:
import pandas as pd
from itertools import product
from sklearn.preprocessing import LabelEncoder

# Set display options for pandas
pd.set_option("display.max_columns", None)

# Load data
df = pd.read_csv("DTM/DTM.csv")

🧹 **Initial Cleaning**


In [ ]:
# Drop columns with too many missing values
df = df.dropna(axis=1, thresh=800)

# Drop unnecessary columns
df = df.drop(columns=['Created', 'Year'])

# Drop rows where 'Vendor' column is missing
df = df.dropna(subset=['Vendor'])

# Standardize 'Shortage Date' column to datetime
df['Shortage Date'] = pd.to_datetime(df['Shortage Date'])

# Fill missing values
df[['Downtime', 'Missed Vehicle']] = df[['Downtime', 'Missed Vehicle']].fillna(0)
df[['LiMa Comment', 'Code 1']] = df[['LiMa Comment', 'Code 1']].fillna('No Data')

📆 **Add Calendar Features**

In [ ]:
# Extract calendar-based features
df['DayOfWeek'] = df['Shortage Date'].dt.dayofweek
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
df['Month'] = df['Shortage Date'].dt.month
df['Quarter'] = df['Shortage Date'].dt.quarter

📊 **Aggregate Daily Data per Vendor**

In [ ]:
# Aggregate data by Vendor and Shortage Date
df = df.groupby(['Vendor', 'Shortage Date']).agg({
    'Downtime': 'sum',
    'Missed Vehicle': 'sum'
}).reset_index()

# Fill missing dates for each vendor and shortage date combination
all_vendors = df['Vendor'].unique()
full_dates = pd.date_range(df['Shortage Date'].min(), df['Shortage Date'].max(), freq='D')
full_index = pd.DataFrame(product(all_vendors, full_dates), columns=['Vendor', 'Shortage Date'])

df = full_index.merge(df, how='left', on=['Vendor', 'Shortage Date'])

# Fill missing downtime and missed vehicle data with 0
df[['Downtime', 'Missed Vehicle']] = df[['Downtime', 'Missed Vehicle']].fillna(0)

🔁 **Add Lag Features & Rolling Averages**

In [ ]:
# Sort data by Vendor and Shortage Date
df = df.sort_values(['Vendor', 'Shortage Date']).reset_index(drop=True)

# Define lags you want
lags = [1, 2, 3, 7]

# For each lag, create lagged features
for lag in lags:
    df[f'Lag_Downtime_{lag}'] = df.groupby('Vendor')['Downtime'].shift(lag)
    df[f'Lag_Misses_{lag}'] = df.groupby('Vendor')['Missed Vehicle'].shift(lag)

# Fill NaN values for lag columns
lag_cols = [f'Lag_Downtime_{l}' for l in lags] + [f'Lag_Misses_{l}' for l in lags]
df[lag_cols] = df[lag_cols].fillna(0)

📆 **Re-Add Calendar Features to Daily Data**

In [ ]:
# Re-add calendar-based features
df['DayOfWeek'] = df['Shortage Date'].dt.dayofweek
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
df['Month'] = df['Shortage Date'].dt.month
df['Quarter'] = df['Shortage Date'].dt.quarter

**Frequency + Severity Ratings**

In [ ]:
# Aggregate total incidents and downtime per supplier
df = df.groupby('Vendor').agg(
    Total_Incidents=('Downtime', lambda x: (x > 0).sum()),  # count of days with downtime > 0
    Total_Downtime=('Downtime', 'sum')
).reset_index()

# Normalize for easy comparison (scale between 0 and 1)
df['Freq_Score'] = df['Total_Incidents'] / df['Total_Incidents'].max()
df['Severity_Score'] = df['Total_Downtime'] / df['Total_Downtime'].max()

# Combined risk score
df['Risk_Score'] = 0.5 * df['Freq_Score'] + 0.5 * df['Severity_Score']

# Merge back to the original dataframe if needed
df = pd.merge(df, df[['Vendor', 'Risk_Score']], on='Vendor', how='left')

In [ ]:
df.info()

**Rolling Averages**

In [ ]:
# Convert 'YearMonth' to a numeric YYYYMM format
df['YearMonth'] = df['Shortage Date'].dt.year * 100 + df['Shortage Date'].dt.month

# Compute monthly average downtime
monthly_avg = df.groupby(['Vendor', 'YearMonth']).agg(
    Avg_Downtime_Month=('Downtime', 'mean')
).reset_index()

# Create ISO week-year column for weekly aggregation
df['YearWeek'] = df['Shortage Date'].dt.strftime('%Y-W%U')

# Sum downtime per vendor per week
df = df.groupby(['Vendor', 'YearWeek']).agg(
    Weekly_Downtime=('Downtime', 'sum')
).reset_index()

# Sort by Vendor and YearWeek before applying rolling window
df = df.sort_values(['Vendor', 'YearWeek'])

# Rolling average of past 12 weeks
df['Avg_Downtime_12w'] = df.groupby('Vendor')['Weekly_Downtime'].transform(
    lambda x: x.shift(1).rolling(window=12, min_periods=1).mean()
)

# Merge weekly rolling averages back to the original dataframe
df = pd.merge(df, df[['Vendor', 'YearWeek', 'Avg_Downtime_12w']], on=['Vendor', 'YearWeek'], how='left')

# Encoding the 'Vendor' column (create new column, do NOT overwrite Vendor)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Vendor_Encoded'] = label_encoder.fit_transform(df['Vendor'])

# Converting 'YearWeek' to a numerical column
df['YearWeek_Num'] = df['Shortage Date'].dt.year * 100 + df['Shortage Date'].dt.isocalendar().week

# Now drop unnecessary columns and create df
df = df.drop(columns=['Vendor', 'Shortage Date', 'YearWeek'])

# Fill nulls in rolling avg column
df['Avg_Downtime_12w'] = df['Avg_Downtime_12w'].fillna(0)

In [ ]:
df.info()

**Model Time Baby...**

In [ ]:
# Data handling
import numpy as np

# Data splitting and preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb

# Metrics
from sklearn.metrics import mean_squared_error, roc_auc_score, confusion_matrix, roc_curve

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

**Initial Data Split**

In [ ]:
# Split data into features and target
X = df.drop(columns=['Downtime', 'Missed Vehicle'])  # Assuming Downtime and Missed Vehicle are your target columns
y = df[['Downtime', 'Missed Vehicle']]  # You might want to predict both, or choose one

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Models Training and GridSearchCV**

In [ ]:
# Random Forest Regressor (Tree Ensemble)
rf = RandomForestRegressor()

rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# XGBoost Regressor
xgb_model = xgb.XGBRegressor()

xgb_params = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Linear Regression
lr = LinearRegression()

lr_params = {
    'fit_intercept': [True, False],
    'normalize': [True, False]
}

# Define GridSearchCV for each model
rf_grid = GridSearchCV(estimator=rf, param_grid=rf_params, cv=5, n_jobs=-1, verbose=2)
xgb_grid = GridSearchCV(estimator=xgb_model, param_grid=xgb_params, cv=5, n_jobs=-1, verbose=2)
lr_grid = GridSearchCV(estimator=lr, param_grid=lr_params, cv=5, n_jobs=-1, verbose=2)

# Train models with GridSearchCV
rf_grid.fit(X_train, y_train)
xgb_grid.fit(X_train, y_train)
lr_grid.fit(X_train, y_train)


**Plots for Analysis**

In [ ]:
# Best hyperparameters for each model
print("Best Random Forest Parameters:", rf_grid.best_params_)
print("Best XGBoost Parameters:", xgb_grid.best_params_)
print("Best Linear Regression Parameters:", lr_grid.best_params_)

# Evaluate models on the test set
rf_score = rf_grid.score(X_test, y_test)
xgb_score = xgb_grid.score(X_test, y_test)
lr_score = lr_grid.score(X_test, y_test)

print(f"Random Forest R²: {rf_score:.4f}")
print(f"XGBoost R²: {xgb_score:.4f}")
print(f"Linear Regression R²: {lr_score:.4f}")

# Plot model comparison
model_names = ['Random Forest', 'XGBoost', 'Linear Regression']
model_scores = [rf_score, xgb_score, lr_score]

plt.bar(model_names, model_scores)
plt.ylabel('R² Score')
plt.title('Model Comparison')
plt.show()


Some next steps:
- Use "BAG OF WORDS" on LIMA comments and codes